In [2]:
import numpy as np
# import networkx as nx
import rustworkx as nx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
# %matplotlib qt


In [3]:
def distribute(needs,space): #distributes cars, there might be a faster way, but i cannot think of one right now
    endresult=np.zeros(len(needs),dtype=int)
    # filter=needs
    while space>=np.sum((needs>0)):
        add=min(np.min(needs[needs>0]),space//np.sum((needs>0)))
        endresult[needs>0]+=add
        space-=add*np.sum((needs>0))
        needs[needs>0]-=add
    endresult[np.nonzero(needs>0)[0][:space]]+=1
    return endresult
class edge(object):
    def __init__(self,length,speed,lanes,id=0):
        self.length = int(length)
        self.lanes = int(lanes)
        self.speed=int(speed)
        self.cars = np.zeros((int(lanes*length/(4.5+55)),5)) #collums: occupied,age,remaining travel time, spawnpoint, destination
        self.id=id
    def __str__(self): 
        return f"edge {self.id} cars:{np.sum(self.cars[:,0])}"
    def __repr__(self): 
        return f"edge {self.id} cars:{np.sum(self.cars[:,0])}"
    def C(self): #capacity of edge
        return int(self.lanes*self.length/(4.5+55))
    def tto(self):
        return self.length/self.speed*60/1000
    def update(self):
        Filter = np.nonzero(self.cars[:,0]==1)[0] #Filter for 
        self.cars[Filter,2] -= 1 #travelled one minute
        self.cars[Filter,1] += 1 #aged one minute
    def tt_avg(self):
        if np.sum(self.cars[:,0])==0:
            return self.tto() #defaults to tt0 if road is empty
        else:
            return (np.sum(self.cars[self.cars[:,2]>0,2])/np.sum(self.cars[:,0])) #otherwise average the traveltime
    def tt(self):
        return self.tto()*(1+0.15*(self.size()/self.C())**4)
    def size(self):
        return sum(self.cars[:,0])
class node(object):
    def __init__(self,input=[],output=[],destinations=[],id=0):
        self.id=id
        self.output = output
        self.input = input
        self.destinations=destinations
        self.waitlist=dict(zip(self.output,[dict(zip(self.input,[np.zeros(0,dtype=int) for j in self.input]))for i in self.output]))
        self.buildup=np.zeros((0,5))
        self.times=np.zeros((0,2)) #array with first collumn as times, and second as origins
    def initwaitlist(self):
        if len(self.waitlist.keys())>0: print("waitlist already had keys",self.waitlist); return
        else: self.waitlist=dict(zip(self.output,[dict(zip(self.input,[np.zeros(0,dtype=int) for j in self.input]))for i in self.output]))
    def __str__(self): 
        if len(self.buildup): return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]} and buildup:{len(self.buildup)}"
        else: return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]}"
    def __repr__(self): 
        if len(self.buildup): return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]} and buildup:{len(self.buildup)}"
        else: return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]}"
    def size(self):
        return len(self.buildup)
    def flow(self):
        for i in self.input:
            Remove=np.nonzero(np.logical_and(i.cars[:,4]==self.id,np.logical_and(i.cars[:,2]<=0,i.cars[:,0]==1)))[0]
            self.times=np.concatenate((self.times,i.cars[Remove,:][:,[1,3]])) #remove all cars that have arrived at their destination
            i.cars[Remove,:]=0
            if self.output==[]:print(f"node {self.id} had no output roads") ;continue
            Filter2 = np.nonzero(np.logical_and(i.cars[:,2]==0,i.cars[:,0]==1))[0] #check which cars arrived now
            # p=np.ones(len(self.output))
            # for index,edges in enumerate(self.output):
            #     if np.sum(edges.cars[:,0])==0: p[index]=1/edges.tto()
            #     else: p[index]=1/(max(np.sum(edges.cars[edges.cars[:,2]>0,2]),1)/np.sum(edges.cars[:,0]))
            # choices=np.random.choice(self.output,len(Filter2),p=p/np.sum(p)) ### can be replaced by larger function to better decide the choices
            choices=np.array([gfd.edges[paths[self.id][endgoal][:2]]["object"] for endgoal in i.cars[Filter2,4]])
            for indexes in choices:
                if indexes not in self.output: print("fuck")
            for j in self.output:
                self.waitlist[j][i]=np.concatenate((self.waitlist[j][i],np.array(Filter2[choices==j],dtype=int))) #add new indexes along with their location at the end
        for i in self.output:
            if sum([len(self.waitlist[i][j]) for j in self.input])==0: continue #doesn't need to do anything if there are no cars waiting
            elif sum([len(self.waitlist[i][j]) for j in self.input])<=np.sum(i.cars[:,0]==0):
                for j in self.input:
                    filter=np.nonzero(i.cars[:,0]==0)[0][:len(self.waitlist[i][j])]
                    i.cars[filter]=j.cars[self.waitlist[i][j],:] #put cars in next edge
                    i.cars[filter,2]=np.maximum(np.ceil(i.tt()+np.random.normal(0,2,size=len(filter))),1)
                    j.cars[self.waitlist[i][j],:]=0 #empty the cars
                    self.waitlist[i][j]=np.zeros(0,dtype=int) #empty waitlist
            else:
                space=np.sum(i.cars[:,0]==0)
                needs=np.array([len(self.waitlist[i][j]) for j in self.input])
                allowances=distribute(needs,space)
                for k,j in enumerate(self.input):
                    filter=np.nonzero(i.cars[:,0]==0)[0][:allowances[k]]
                    i.cars[filter]=j.cars[self.waitlist[i][j][:allowances[k]]] ###probs dont need [filtertje,:] [filtertje] seems enough
                    i.cars[filter,2]=np.maximum(np.ceil(i.tt()+np.random.normal(0,2,size=len(filter))),1)
                    j.cars[self.waitlist[i][j][:allowances[k]],:]=0 #empty the cars
                    self.waitlist[i][j]=self.waitlist[i][j][allowances[k]:]
    def generate(self,state,mean=85,sd=1.5,destinations=[2]): ##change code to account for more generating cities and differing distributions
        N=int(np.random.normal(mean,sd)+0.5) #+.5 rounds it to the nearest integer
        newcars=np.array([np.ones(N),np.zeros(N),np.random.normal(0,2,size=N),np.full(N,self.id),np.random.choice(self.destinations,N)]).transpose()
        newcars=np.concatenate((self.buildup,newcars)) #needs to handle the built up cars first
        edgelists=np.array([gfd.edges[paths[self.id][endgoal][:2]]["object"] for endgoal in newcars[:,4]])
        for i in edgelists:
            if i not in self.output: print("fuck")
        self.buildup=np.zeros((0,5))
        for i in self.output:
            going=edgelists==i
            filter = np.nonzero(i.cars[:,0]==0)[0]
            # print(np.sum(i.cars[:,0]==0),np.sum(going))
            if np.sum(i.cars[:,0]==0)>=np.sum(going): #can go if there are enough spaces for it
                # print("this case")
                i.cars[filter[:np.sum(going)],:]=newcars[going,:]
                i.cars[filter[:np.sum(going)],2]=np.maximum(np.ceil(i.tt()+i.cars[filter[:np.sum(going)],2]),1)
            else: 
                spaces=np.sum(i.cars[:,0]==0)
                i.cars[filter,:]=newcars[np.nonzero(going==1)[0][:spaces],:]
                i.cars[filter,2]=np.maximum(np.ceil(i.tt()+i.cars[filter,2]),1)
                build=newcars[np.nonzero(going==1)[0][spaces:]] 
                build[:,1]+=1 #update age of cars that stayed in the city
                self.buildup=np.concatenate((self.buildup,build)) #throw all excess cars into the buildup
        

In [4]:
nodes_gpd = gpd.read_file('./wegengraaf2/points.shp')
lines_gpd = gpd.read_file('./wegengraaf2/lines.shp')

In [5]:
lines_gpd["capacity"]=lines_gpd['length']*lines_gpd['nbanen']*1/(4.5+55)
nodes_gpd["is_auto_br"][np.isnan(nodes_gpd["is_auto_br"])]=0
nodes_gpd["spawn_hoev"][np.isnan(nodes_gpd["spawn_hoev"])]=0

C:\Users\Alexander\AppData\Local\Temp\ipykernel_9660\414636931.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  nodes_gpd["is_auto_br"][np.isnan(nodes_gpd["is_auto_br"])]=0
C:\Users\Alexander\AppData\Local\Temp\ipykernel_9660\414636931.py

In [34]:
gfd = nx.PyDiGraph()

class GraphNode:
    def __init__(self, nodeID, x, y, stad_naam, is_auto_br, spawn_hoev):
        self.nodeID = nodeID
        self.x = x
        self.y = y
        self.stad_naam = stad_naam
        self.is_auto_br = is_auto_br
        self.spawn_hoev = spawn_hoev

class GraphEdge:
    def __init__(self, node_start, node_end, wegnr, length, maxshd, nbanen):
        self.node_start = node_start
        self.node_end = node_end
        self.wegnr = wegnr
        self.length = length
        self.maxshd = maxshd
        self.nbanen = nbanen

    def __getitem__(self, key):
        match key:
            case 'node_start':
                return self.node_start
            case 'node_end':
                return self.node_end
            case 'wegnr':
                return self.wegnr
            case 'length':
                return self.length
            case 'maxshd':
                return self.maxshd
            case 'nbanen':
                    return self.nbanen

# graph_indices[nodeID] = index of graph
graph_indices = {}

for _, k in nodes_gpd.iterrows():
    nodeID = int(k['nodeID'])
    ind = gfd.add_node(GraphNode(nodeID,
                k['geometry'].x,
                k['geometry'].y,
                k['stad_naam'],
                k['is_auto_br'],
                k['spawn_hoev']))
    graph_indices[nodeID] = ind

for _, row in lines_gpd.iterrows():
    node_start = int(row['node_start'])
    node_end = int(row['node_end'])
    gfd.add_edge(graph_indices[node_start], graph_indices[node_end],
                GraphEdge(node_start, node_end,
                    row['wegnr'],
                    row['length'],
                    row['maxshd'],
                    row['nbanen']))
    

In [24]:
from rustworkx import visualization

In [13]:
def weight_edge_min(f, t, attr):
    return attr['length']/attr['maxshd']/1000*60

def weight_edge_dist(f, t, attr):
    return attr['length']

In [80]:
class DijkstraPathDict:
    def __init__(self, paths):
        self.paths = paths
    
    def __getitem__(self, key): # node ID, not index
        temp_dict = dict(self.paths[graph_indices[key]])
        return temp_dict
        # return {graph_indices[n]: temp_dict[n] for n in temp_dict.keys()}

In [81]:
%%time
dist = {}
paths = {}

def all_pairs_dijkstra(graph, weight):
    distances = nx.all_pairs_dijkstra_path_lengths(gfd, lambda x: weight(None, None, x))
    paths = nx.all_pairs_dijkstra_shortest_paths(gfd, lambda x: weight(None, None, x))
    return DijkstraPathDict(paths)

d = all_pairs_dijkstra(gfd, weight=weight_edge_min)

# for (n, (a, b)) in nx.all_pairs_dijkstra(gfd, weight=weight_edge_min):
#     dist[n] = a
#     paths[n] = b    

CPU times: total: 0 ns
Wall time: 0 ns


In [84]:
d[1000].keys()

dict_keys([97, 163, 189, 5, 190, 192, 241, 162, 135, 242, 136, 208, 225, 237, 193, 194])

In [8]:
##this can be used to look only at a subgraph consisting of shortest paths between cities
# path1,path2,path3=paths[1000][1001],paths[1001][1002],paths[1000][1002]
# gfd = nx.DiGraph(gfd.subgraph(path1+path2+path3))

### shortest path examples

In [9]:
from_node = 1000
to_node = 1001

In [10]:
(pair_dist, pair_path) = nx.single_source_dijkstra(gfd, from_node, to_node, weight=weight_edge_min)
print(pair_dist, pair_path)

19.354978988448625 [1000, 6, 204, 249, 206, 192, 193, 146, 1001]


In [11]:
# plt.figure(figsize=(20, 20))
# nx.draw(gfd, pos={n: [gfd.nodes[n]['x'], gfd.nodes[n]['y']] for n in list(gfd.nodes)}, with_labels=True, node_shape='s')
# nx.draw(subgr, pos={n: [gfd.nodes[n]['x'], gfd.nodes[n]['y']] for n in list(gfd.nodes)})

path = paths[from_node][to_node]
subgr = nx.DiGraph(gfd.subgraph(path))
# nx.draw_networkx_edges(subgr, pos={n: [gfd.nodes[n]['x'], gfd.nodes[n]['y']] for n in list(gfd.nodes)}, edge_color='red', width=3)

# plt.show()

### Running the model

In [12]:
sources=nodes_gpd["nodeID"][np.nonzero(nodes_gpd["is_auto_br"])[0]]
# sources=[1000,1001,1002]
nx.set_edge_attributes(gfd,{e: edge(gfd.edges[e]["length"],gfd.edges[e]["maxshd"],gfd.edges[e]["nbanen"],id=e) for e in gfd.edges},"object") #put edge objects on the edges
nx.set_node_attributes(gfd,{n: node(input=[gfd.edges[e]["object"] for e in gfd.in_edges(n)],output=[gfd.edges[e]["object"] for e in gfd.out_edges(n)],id=n) for n in gfd.nodes},"object") 
#put node objects on nodes
for i in sources:
    gfd.nodes[i]["object"].destinations=list(filter(lambda x:x!=i,sources)) #destination is any city except source

In [46]:
def weight_capacity(f, t, attr):
    cars = np.sum(attr['object'].cars[:, 0])
    capacity = attr['object'].C()
    return attr['length']/attr['maxshd']/1000*60 * (1 + cars/capacity)

In [47]:
sources=nodes_gpd["nodeID"][np.nonzero(nodes_gpd["is_auto_br"])[0]]
# sources=[1000,1001,1002]

minutes_init=0 #initialisation time
minutes_sim=60 #simulation time
cars_forw=np.zeros((len(lines_gpd),minutes_sim))
cars_backw=np.zeros((len(lines_gpd),minutes_sim))
def tt_avg(f, t, attr):
    return attr['object'].tt_avg()
for minute in range(minutes_init):
    paths={n: b for (n, (a, b)) in nx.all_pairs_dijkstra(gfd, weight=weight_capacity)}
    for i in sources:
        # print(i,gfd.nodes[i]["object"])
        gfd.nodes[i]["object"].generate('') #generate cars at all sources
    for e in gfd.edges:
        # print(e,gfd.edges[e]["object"])
        gfd.edges[e]["object"].update()
    for n in gfd.nodes:
        # print(n,gfd.nodes[n]["object"])
        gfd.nodes[n]["object"].flow()
for minute in range(minutes_sim):
    print(f"{minute}", end=", ")
    paths={n: b for (n, (a, b)) in nx.all_pairs_dijkstra(gfd, weight=weight_capacity)}
    for i in sources:
    # print(i,gfd.nodes[i]["object"])
        gfd.nodes[i]["object"].generate('') #generate cars at all sources
    for e in gfd.edges:
        # print(e,gfd.edges[e]["object"])
        gfd.edges[e]["object"].update()
    for n in gfd.nodes:
        # print(n,gfd.nodes[n]["object"])
        gfd.nodes[n]["object"].flow()
    for index,(start,end) in enumerate(zip(lines_gpd["node_start"],lines_gpd["node_end"])):
        # if (start,end) in gfd.edges:
        cars_forw[index,minute]=gfd.edges[(start,end)]["object"].size()
        cars_backw[index,minute]=gfd.edges[(end,start)]["object"].size()
        # else: continue

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 

KeyboardInterrupt: 

In [ ]:
cars_forw_pd=pd.DataFrame(cars_forw,columns=[f"f{i}" for i in range(minutes_sim)])
cars_backw_pd=pd.DataFrame(cars_backw,columns=[f"b{i}" for i in range(minutes_sim)])
lines_gpd=pd.concat((lines_gpd,pd.DataFrame(cars_forw_pd),cars_backw_pd),axis=1)

In [14]:
print(sources)

245    1000.0
246    1001.0
247    1002.0
252    1003.0
253    1004.0
254    1005.0
255    1006.0
256    1007.0
257    1008.0
258    1009.0
259    1010.0
260    1011.0
Name: nodeID, dtype: float64


In [15]:
gfd.edges[(1002,240)]

{'wegnr': 'Sneek Noord',
 'length': 2114.123792163208,
 'maxshd': 50,
 'nbanen': 1,
 'object': edge (1002, 240) cars:19.0}

In [16]:
# lines_gpd

In [17]:
def gen_text(string):
    row_index = traffic_map.index[traffic_map['name'] == string]
    if "City" in string:
        return "City " + string[-1]
    elif 'Node' in string:
        if row_index!=None:
            num_of_car = traffic_map.loc[row_index[0], 'cars']
            return '\n' + string[-1] + f'\n cars: {num_of_car}'
        else:
            return '\n' + string[-1] + '\n cars: Not found'
    elif 'Edge' in string:
        if row_index!=None:
            num_of_car = traffic_map.loc[row_index[0], 'cars']
            return f'cars: {num_of_car}'
        else:
            return 'cars: Not found'
    else:
        return ''
def gen_text_lines(index):
    return lines_gpd["wegnr"][index]+"\n"+str(lines_gpd["cars"][index])



In [18]:
nodes_gpd

,x,y,nodeID,stad_naam,is_auto_br,spawn_hoev,geometry
0,634450.287500,7.034872e+06,0.0,None,0.0,0.0,POINT (634450.288 7034871.712)
1,631261.700000,7.034494e+06,1.0,None,0.0,0.0,POINT (631261.7 7034493.5)
2,678454.257259,6.976978e+06,2.0,None,0.0,0.0,POINT (678454.257 6976977.584)
3,675386.000000,6.974835e+06,3.0,None,0.0,0.0,POINT (675386 6974834.7)
4,673755.301869,6.973613e+06,4.0,None,0.0,0.0,POINT (673755.302 6973612.649)
...,...,...,...,...,...,...,...
256,617080.349435,6.170803e+05,1007.0,Franeker,1.0,0.0,POINT (617080.349 7017852.729)
257,667774.750484,6.677748e+05,1008.0,Dokkum,1.0,0.0,POINT (667774.75 7043345.249)
258,635511.580819,6.355116e+05,1009.0,Lemmer,1.0,0.0,POINT (635511.581 6954311.894)
259,615308.511335,6.153085e+05,1010.0,Bolsward,1.0,0.0,POINT (615308.511 6995205.261)


In [19]:
# i=59
# lines_gpd["cars"] = lines_gpd[f"f{i}"]
# lines_gpd["textpromt"]=lines_gpd.apply(gen_text_lines,axis=0)

In [20]:
lines_gpd

,wegnr,length,maxshd,node_start,node_end,nbanen,geometry,capacity,f0,f1,...,b50,b51,b52,b53,b54,b55,b56,b57,b58,b59
0,N393,1924.333925,80,0,1,1,"LINESTRING (634450.288 7034871.712, 634130.3 7...",32.341747,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,N393,1660.777370,50,0,86,1,"LINESTRING (636926.484 7034136.391, 635716.5 7...",27.912225,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,N380,2271.218173,60,2,3,1,"LINESTRING (678454.257 6976977.584, 678365.673...",38.171734,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,N380,1177.078109,60,2,216,1,"LINESTRING (678454.257 6976977.584, 678483.198...",19.782825,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,N353,4651.621140,30,2,12,1,"LINESTRING (682722.705 6970661.324, 682721.7 6...",78.178507,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,Bolsward West,1580.854233,50,1010,57,1,"LINESTRING (613403.016 6994234.021, 613602.41 ...",26.568979,8.0,8.0,...,15.0,18.0,19.0,21.0,20.0,17.0,20.0,20.0,19.0,19.0
308,Burgum Zuid,1555.615683,50,1011,205,1,"LINESTRING (666518.887 7018843.045, 666711.848...",26.144801,20.0,22.0,...,18.0,23.0,16.0,18.0,20.0,19.0,23.0,16.0,15.0,21.0
309,Burgum-Noord,1557.198917,50,1011,156,1,"LINESTRING (666493.158 7018830.181, 666403.11 ...",26.171410,5.0,19.0,...,3.0,2.0,1.0,1.0,3.0,3.0,2.0,0.0,0.0,1.0
310,Friesland Zuid 2,4683.209945,50,1000,175,1,"LINESTRING (645406.714 7012605.739, 644955.789...",78.709411,37.0,77.0,...,3.0,2.0,9.0,8.0,8.0,7.0,5.0,6.0,6.0,3.0


In [21]:
lines_gpd[f"f0"]

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
307     8.0
308    20.0
309     5.0
310    37.0
311     0.0
Name: f0, Length: 312, dtype: float64

In [22]:
lines_gpd[f"f{0}"]/lines_gpd["capacity"]

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
307    0.301103
308    0.764970
309    0.191048
310    0.470084
311    0.000000
Length: 312, dtype: float64

In [23]:
import contextily as cx

In [24]:
cities = gpd.read_file('./wegengraaf2/cities.shp').to_crs(epsg=3857)

In [25]:
from matplotlib.animation import FFMpegWriter


# plt.figure(figsize=(11,12), dpi=201.5)
# ax = plt.subplot()


def update(i):
    ax = plt.subplot()
    ax.set_title(f"Frame {i}")
    lines_gpd.plot(ax=ax, column=lines_gpd[f"f{i}"]/lines_gpd["capacity"],
                    cmap='autumn',
                    linewidth=4,
                    # legend=True,
                    # legend_kwds={'label': 'Aantal banen', 'shrink': 0.6},
                    zorder=1)
    nodes_gpd.plot(ax=ax, markersize=10, color='blac+k', zorder=2)
    cities.plot(ax=ax, 
            column='population',)
    cx.add_basemap(ax)
    ax.savefig(f'./img/{i}.png')

for k in range(0, 60):
    update(k)


# anim = FuncAnimation(ax.get_figure(), update, frames=2)
# writervideo = FFMpegWriter(fps=1) 
# anim.save('./friesland.mp4', writer=writervideo,dpi = 201.5) 

ValueError: 'color' kwarg must be a color or sequence of color specs.  For a sequence of values to be color-mapped, use the 'c' argument instead.

In [105]:
# traffic_map["cars"] = data1[:,i]
# traffic_map['textpromt'] = traffic_map['name'].apply(gen_text)
fig, ax = plt.subplots(1, figsize=(15, 8),dpi = 200)
cmap = plt.cm.get_cmap('autumn', 120)
cmap2 = plt.cm.get_cmap('Greens', 120)

nodes_gpd.plot( linewidth=0.1, ax=ax, edgecolor=".4")

def update(i):
    ax.clear()
    lines_gpd.plot(ax=ax, column=lines_gpd[f"f{i}"]/lines_gpd["capacity"],
                cmap='Accent',
                linewidth=3,
                # legend=True,
                # legend_kwds={'label': 'Aantal banen', 'shrink': 0.6},
                zorder=1)
    nodes_gpd.plot(ax=ax, markersize=25, color='black', zorder=2)


    # ax.clear()
    
    #plotting edges colored by fractions of capacity
    # lines_gpd.plot(column=lines_gpd[f"f{i}"]/lines_gpd["capacity"], cmap=cmap, linewidth=0.4, ax=ax,vmin=0,vmax=1)
    # lines_gpd.apply(lambda x: ax.annotate(text=f"{x['f59']/x['capacity']:.2f}" , xy=x.geometry.centroid.coords[0], ha='center') if x['f59']/x['capacity']>0.9 else 0, axis=1)
anim = FuncAnimation(fig,update,interval = 100)

#create a colorr barr
cax = fig.add_axes([0.92, 0.1, 0.02, 0.78])  # Adjust the position and size as needed
bar_info = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=100))
bar_info._A = [] ### geen idee wat dit doet
cbar = fig.colorbar(bar_info, cax=cax)


C:\Users\Alexander\AppData\Local\Temp\ipykernel_15624\3702069414.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('autumn', 120)
C:\Users\Alexander\AppData\Local\Temp\ipykernel_15624\3702069414.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap2 = plt.cm.get_cmap('Greens', 120)
C:\Users\Alexander\AppData\Local\Temp\ipykernel_15624\3702069414.py:25: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data

In [106]:
print(sources)
for i in sources:
    print(len(gfd.nodes[i]["object"].times))
    print(len(gfd.nodes[i]["object"].buildup))

245    1000.0
246    1001.0
247    1002.0
252    1003.0
253    1004.0
254    1005.0
255    1006.0
256    1007.0
257    1008.0
258    1009.0
259    1010.0
260    1011.0
Name: nodeID, dtype: float64
369
3068
343
3978
429
3835
242
3677
293
4447
206
4007
402
4427
292
4124
402
3922
159
4602
656
3575
378
4208
